In [1]:
from omegaconf import OmegaConf
from deepnote import MusicRepr
from importlib import reload

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

seed_everything(42)

42

## Config

In [2]:
conf = OmegaConf.load('conf.yaml')

## Dataset

In [3]:
from midi_transformer import LMDataset, get_dataloaders


dataset = LMDataset(
    **conf['data']
)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [4]:
train_loader, val_loader = get_dataloaders(dataset, batch_size=24, n_jobs=4, val_frac=0.1)

train dataset has 8972006 samples and val dataset has 996889 samples.


In [5]:
x, y = dataset[0]
x.shape, y.shape

((512, 8), (512, 8))

In [6]:
for b in val_loader:
    for k in b:
        print(k, b[k].shape)
    break

X torch.Size([24, 512, 8])
X_len torch.Size([24])
labels torch.Size([24, 512, 8])


## Model

In [7]:
from midi_transformer import CPTransformer

model = CPTransformer(conf['model'])
print('model has', model.count_parameters(), 'parameters.')

model has 17322172 parameters.


In [8]:
# model.step(b)

## Trainer

In [9]:
name = '-'.join(conf['data']['instruments'])
print('model name:',name)

logger = TensorBoardLogger(save_dir='logs/', name=name)
lr_logger = LearningRateMonitor(logging_interval='step')
checkpoint = ModelCheckpoint(dirpath=f'weights/{name}/', 
                             filename='{epoch}-{val_loss:.2f}', 
                             monitor='val_loss',
                             save_top_k=10, 
                             period=1)

trainer = Trainer(benchmark=True, 
                  gpus=1, 
#                   reload_dataloaders_every_epoch=True,
#                   gradient_clip_val=0.5,
                  accumulate_grad_batches=1,
                  logger=logger, 
                  max_epochs=conf['model']['max_epochs'],
                  callbacks=[checkpoint, lr_logger])


/home/soroosh/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Checkpoint directory weights/piano-bass-drums-guitar-ensemble/ exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


model name: piano-bass-drums-guitar-ensemble


In [ ]:
trainer.fit(model, train_loader, val_loader)


  | Name      | Type             | Params
-----------------------------------------------
0 | criterion | CrossEntropyLoss | 0     
1 | emb       | CPEmbedding      | 276 K 
2 | encoder   | Encoder          | 16.8 M
3 | head      | CPHeadLayer      | 222 K 
-----------------------------------------------
17.3 M    Trainable params
0         Non-trainable params
17.3 M    Total params


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## generation

In [ ]:
gen = model.generate(
    prompt=None,
    max_len=10, 
    temperatures={
        'ttype' : 1.5,
        'position': 0.9,
        'tempo': 0.5,
        'chord': 0.8,
        'instrument': 1.,
        'pitch': 1.1,
        'duration': 0.8,
        'velocity': 0.8
    }
)
seq = MusicRepr.from_cp(gen)
seq.to_midi('assets/gen.mid')